In [1]:
import pandas as pd
import glob
from sqlalchemy import create_engine
import psycopg2


# Data

In [2]:
tripdata_files = glob.glob("data/JC-*-citibike-tripdata.csv")

### Citibike data

In [3]:
df_list = []
for filename in tripdata_files:
  data = pd.read_csv(filename)
  df_list.append(data)

citibike_tripdata = pd.concat(df_list)

### Weather data

In [4]:
weather_data = pd.read_csv("data/newark_airport_2016.csv")

# Citibike - Exploratory Data Analysis

In [5]:
print(citibike_tripdata.head())

   Trip Duration           Start Time            Stop Time  Start Station ID  \
0            361  2016-02-01 00:31:18  2016-02-01 00:37:19              3202   
1            297  2016-02-01 01:55:05  2016-02-01 02:00:02              3195   
2           1155  2016-02-01 02:40:05  2016-02-01 02:59:20              3183   
3           1769  2016-02-01 05:11:28  2016-02-01 05:40:58              3214   
4            935  2016-02-01 05:48:24  2016-02-01 06:03:59              3203   

  Start Station Name  Start Station Latitude  Start Station Longitude  \
0       Newport PATH               40.727224               -74.033759   
1            Sip Ave               40.730743               -74.063784   
2     Exchange Place               40.716247               -74.033459   
3   Essex Light Rail               40.712774               -74.036486   
4      Hamilton Park               40.727596               -74.044247   

   End Station ID  End Station Name  End Station Latitude  \
0            3203  

In [6]:
print(citibike_tripdata.info())
print(citibike_tripdata.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 247584 entries, 0 to 19487
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Trip Duration            247584 non-null  int64  
 1   Start Time               247584 non-null  object 
 2   Stop Time                247584 non-null  object 
 3   Start Station ID         247584 non-null  int64  
 4   Start Station Name       247584 non-null  object 
 5   Start Station Latitude   247584 non-null  float64
 6   Start Station Longitude  247584 non-null  float64
 7   End Station ID           247584 non-null  int64  
 8   End Station Name         247584 non-null  object 
 9   End Station Latitude     247584 non-null  float64
 10  End Station Longitude    247584 non-null  float64
 11  Bike ID                  247584 non-null  int64  
 12  User Type                247204 non-null  object 
 13  Birth Year               228585 non-null  float64
 14  Gender    

In [7]:
print(citibike_tripdata.duplicated())

0        False
1        False
2        False
3        False
4        False
         ...  
19483    False
19484    False
19485    False
19486    False
19487    False
Length: 247584, dtype: bool


# Citibike - data preparation

- remove unwanted columns
- fix Null entries
- Change Birth year entries to integer instead of float
- Split start and stop time columns
- Rename columns

In [8]:
citibike_tripdata = citibike_tripdata[['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', #'Start Station Latitude',
       #'Start Station Longitude',
       'End Station ID',
       'End Station Name',
       #'End Station Latitude', 'End Station Longitude',
       'Bike ID', 'User Type',
       'Birth Year', 'Gender']].copy()

In [9]:
# Birth year column
# first NaN must be replaced with 0
# change float to integer

citibike_tripdata['Birth Year'] = citibike_tripdata['Birth Year'].fillna(0)
citibike_tripdata['Birth Year'] = citibike_tripdata['Birth Year'].astype(int)

In [10]:
#user type column
# Replce NaN with 'Unknown'

citibike_tripdata['User Type'] = citibike_tripdata['User Type'].fillna('Unknown')

In [11]:
print(citibike_tripdata.info())

<class 'pandas.core.frame.DataFrame'>
Index: 247584 entries, 0 to 19487
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Trip Duration       247584 non-null  int64 
 1   Start Time          247584 non-null  object
 2   Stop Time           247584 non-null  object
 3   Start Station ID    247584 non-null  int64 
 4   Start Station Name  247584 non-null  object
 5   End Station ID      247584 non-null  int64 
 6   End Station Name    247584 non-null  object
 7   Bike ID             247584 non-null  int64 
 8   User Type           247584 non-null  object
 9   Birth Year          247584 non-null  int64 
 10  Gender              247584 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 22.7+ MB
None


In [12]:
# Split "Start Time" column

citibike_tripdata[['start_date', 'start_time']] = citibike_tripdata['Start Time'].str.split(' ', expand=True)

In [13]:
print(citibike_tripdata.columns)

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'End Station ID', 'End Station Name', 'Bike ID',
       'User Type', 'Birth Year', 'Gender', 'start_date', 'start_time'],
      dtype='object')


In [14]:
# Split "Stop Time" column

citibike_tripdata[['stop_date', 'stop_time']] = citibike_tripdata['Stop Time'].str.split(' ', expand=True)

In [15]:
# Rename columns
# 'Start Time' column becomes start_date_time, because of the date and time combined it is a perfect Primary Key

citibike_tripdata = citibike_tripdata.rename(columns={
    'Trip Duration': 'trip_duration',
    'Start Time': 'start_date_time',
    'Start Station ID': 'start_station_id',
    'Start Station Name': 'start_station_name',
    'End Station Name': 'end_station_name',
    'End Station ID': 'end_station_id',
    'Bike ID': 'bike_id',
    'User Type': 'user_type',
    'Birth Year': 'birth_year',
    'Gender': 'gender'
    })

In [16]:
# remove old stop time columns

citibike_tripdata = citibike_tripdata[['trip_duration', 'start_date_time', #'Stop Time',
                                        'start_station_id',
       'start_station_name', 'end_station_id', 'end_station_name', 'bike_id',
       'user_type', 'birth_year', 'gender', 'start_date', 'start_time',
       'stop_date', 'stop_time']]

In [17]:
print(citibike_tripdata.columns)

Index(['trip_duration', 'start_date_time', 'start_station_id',
       'start_station_name', 'end_station_id', 'end_station_name', 'bike_id',
       'user_type', 'birth_year', 'gender', 'start_date', 'start_time',
       'stop_date', 'stop_time'],
      dtype='object')


In [18]:
new_citibike_tripdata = citibike_tripdata

# Weather - Exploratory Data Analysis

From data documentation:\
Note: 9’s in a field (e.g.9999) indicate missing data or data that has not been received.\
The five core values are:

- PRCP = Precipitation (mm or inches as per user preference, inches to hundredths on Daily Form pdf file)
- SNOW = Snowfall (mm or inches as per user preference, inches to tenths on Daily Form pdf file)
- SNWD = Snow depth (mm or inches as per user preference, inches on Daily Form pdf file)
- TMAX = Maximum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on
Daily Form pdf file
- TMIN = Minimum temperature (Fahrenheit or Celsius as per user preference, Fahrenheit to tenths on
Daily Form pdf file

In [19]:
print(weather_data.head())

       STATION                                         NAME        DATE  \
0  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-01   
1  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-02   
2  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-03   
3  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-04   
4  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-05   

    AWND  PGTM  PRCP  SNOW  SNWD  TAVG  TMAX  TMIN  TSUN  WDF2   WDF5  WSF2  \
0  12.75   NaN   0.0   0.0   0.0    41    43    34   NaN   270  280.0  25.9   
1   9.40   NaN   0.0   0.0   0.0    36    42    30   NaN   260  260.0  21.0   
2  10.29   NaN   0.0   0.0   0.0    37    47    28   NaN   270  250.0  23.9   
3  17.22   NaN   0.0   0.0   0.0    32    35    14   NaN   330  330.0  25.9   
4   9.84   NaN   0.0   0.0   0.0    19    31    10   NaN   360  350.0  25.1   

   WSF5  
0  35.1  
1  25.1  
2  30.0  
3  33.1  
4  31.1  


In [20]:
print(weather_data.info())
print(weather_data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  366 non-null    object 
 1   NAME     366 non-null    object 
 2   DATE     366 non-null    object 
 3   AWND     366 non-null    float64
 4   PGTM     0 non-null      float64
 5   PRCP     366 non-null    float64
 6   SNOW     366 non-null    float64
 7   SNWD     366 non-null    float64
 8   TAVG     366 non-null    int64  
 9   TMAX     366 non-null    int64  
 10  TMIN     366 non-null    int64  
 11  TSUN     0 non-null      float64
 12  WDF2     366 non-null    int64  
 13  WDF5     364 non-null    float64
 14  WSF2     366 non-null    float64
 15  WSF5     364 non-null    float64
dtypes: float64(9), int64(4), object(3)
memory usage: 45.9+ KB
None
STATION      0
NAME         0
DATE         0
AWND         0
PGTM       366
PRCP         0
SNOW         0
SNWD         0
TAVG         0
TMAX    

Columns with Null entries:
- PGTM (complete column)
- TSUN (complete column)
- WDF5 (2 Null entries)
- WSF5 (2 Null entries)

In [21]:
print(weather_data.duplicated())

0      False
1      False
2      False
3      False
4      False
       ...  
361    False
362    False
363    False
364    False
365    False
Length: 366, dtype: bool


In [22]:
# search for use of 9999 as value

weather_data.eq(9999).any(axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
361    False
362    False
363    False
364    False
365    False
Length: 366, dtype: bool

# Weather - data preparation

- remove unwanted columns
- Split NAME column
- fix Null entries
- Rename columns

In [23]:
# remove columns 
weather_data = weather_data[['STATION', 'NAME', 'DATE', 'AWND', #'PGTM',
                             'PRCP', 'SNOW', 'SNWD',
       'TAVG', 'TMAX', 'TMIN', #'TSUN',
       #'WDF2', 'WDF5', 'WSF2', 'WSF5'
       ]].copy()

In [24]:
# split NAME column

weather_data[['NAME', 'STATE']] = weather_data['NAME'].str.split(',', expand=True)
print(weather_data.head(3))

       STATION                                  NAME        DATE   AWND  PRCP  \
0  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT  2016-01-01  12.75   0.0   
1  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT  2016-01-02   9.40   0.0   
2  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT  2016-01-03  10.29   0.0   

   SNOW  SNWD  TAVG  TMAX  TMIN   STATE  
0   0.0   0.0    41    43    34   NJ US  
1   0.0   0.0    36    42    30   NJ US  
2   0.0   0.0    37    47    28   NJ US  


In [25]:
weather_data.columns

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG',
       'TMAX', 'TMIN', 'STATE'],
      dtype='object')

In [26]:
# Rename columns
weather_data = weather_data.rename(columns={
   'STATION': 'station_id',
   'NAME': 'station_name',
   'DATE': 'date',
   'AWND': 'average_wind_speed',
   'PRCP': 'precipitation',
   'SNOW': 'snowfall',
   'SNWD': 'snow_depth',
   'TAVG': 'average_temp',
   'TMAX': 'max_temp',
   'TMIN': 'min_temp',
   'STATE': 'state'
    })

In [27]:
weather_data.columns

Index(['station_id', 'station_name', 'date', 'average_wind_speed',
       'precipitation', 'snowfall', 'snow_depth', 'average_temp', 'max_temp',
       'min_temp', 'state'],
      dtype='object')

In [28]:
new_weather_data = weather_data

# Database

In [29]:
engine = create_engine('postgresql+psycopg2://thomasdewit:@localhost:5432/citibike_database')

In [ ]:
# citibike database

new_citibike_tripdata.to_sql('citibike_tripdata', engine, if_exists='replace', index=False)

In [ ]:
# weather database

new_weather_data.to_sql('weather_data', engine, if_exists='replace', index=False)

# Data analysis

Possible SQL views:
- daily rides and weather combined
- Most popular Start/End Stations
- Most popular station combo's/routes
- most popular time of year
- Bike rides per hour of day
- Overview of birth years

In [34]:
# daily rides and weather combined

view_daily_ride_weather = 'daily_ride_weather'
query = f'SELECT * FROM {view_daily_ride_weather}'

daily_ride_weather = pd.read_sql(query, engine)

In [43]:
daily_ride_weather.head()

,start_date,count,date,average_wind_speed,precipitation,average_temp
0,2016-01-01,163,2016-01-01,12.75,0.0,41
1,2016-01-02,207,2016-01-02,9.40,0.0,36
2,2016-01-03,276,2016-01-03,10.29,0.0,37
3,2016-01-04,286,2016-01-04,17.22,0.0,32
4,2016-01-05,273,2016-01-05,9.84,0.0,19


In [31]:
# The code above was not scalable, since I need to fetch data from 8 more views.
# So I made a function instead.

def fetch_data_from_views(engine, view):
    query = f'SELECT * FROM {view}'
    dataframe = pd.read_sql(query, engine)
    return dataframe



In [37]:
popular_start_station = 'popular_start_station'
print(fetch_data_from_views(engine, popular_start_station).head())

   start_station_count start_station_name
0                28736      Grove St PATH
1                19014     Exchange Place
2                17137            Sip Ave
3                15300      Hamilton Park
4                13363       Newport PATH


In [38]:
popular_end_station = 'popular_end_station'
print(fetch_data_from_views(engine, popular_end_station).head())

   end_station_count end_station_name
0              38295    Grove St PATH
1              22236   Exchange Place
2              15881          Sip Ave
3              15418    Hamilton Park
4              13533     Newport PATH


In [39]:
popular_station_combos = 'popular_station_combos'
print(fetch_data_from_views(engine, popular_station_combos).head())

  start_station_name end_station_name  combination_count
0      Hamilton Park    Grove St PATH               5742
1       Brunswick St    Grove St PATH               5553
2    McGinley Square          Sip Ave               4728
3      Grove St PATH    Hamilton Park               4135
4     Van Vorst Park    Grove St PATH               3954


In [40]:
popular_month = 'popular_month'
print(fetch_data_from_views(engine, popular_month).head())

                       month  count
0  2016-01-01 00:00:00+01:00   7479
1  2016-02-01 00:00:00+01:00   8250
2  2016-03-01 00:00:00+01:00  13511
3  2016-04-01 00:00:00+02:00  16342
4  2016-05-01 00:00:00+02:00  19488


In [41]:
rides_hour = 'rides_hour'
print(fetch_data_from_views(engine, rides_hour).head())

             hour  count
0 0 days 00:00:00   2533
1 0 days 01:00:00   1316
2 0 days 02:00:00    802
3 0 days 03:00:00    514
4 0 days 04:00:00    580


In [42]:
birth_years = 'birth_years'
print(fetch_data_from_views(engine, birth_years).head())

   birth_year  count
0           0  18999
1        1900      1
2        1934      1
3        1937      4
4        1940      3
